In [1]:
import os
# "jax", "tensorflow", or "torch"
os.environ["KERAS_BACKEND"] = "tensorflow"

In [2]:
import keras
import tensorflow as tf
import tensorflow.experimental.numpy as np

import bayesflow.experimental as bf

2024-04-17 18:42:43.648581: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-17 18:42:43.649086: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 18:42:43.651679: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 18:42:43.682981: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 18:42:44.306816: W tensorflow/compiler/tf2tensorrt/util

In [3]:
@bf.distribution
def two_moons_context():
    r = 0.1 + 0.01 * np.random.standard_normal(size=(1,))
    alpha = np.random.uniform(-0.5 * np.pi, 0.5 * np.pi, size=(1,))
    return dict(r=r, alpha=alpha)

In [4]:
@bf.distribution
def two_moons_prior():
    theta = np.random.uniform(-1.0, 1.0, size=(2,))
    return dict(theta=theta)

In [5]:
@bf.distribution
def two_moons_likelihood(r, alpha, theta):
    x1 = -np.abs(theta[0] + theta[1]) / np.sqrt(2.0) + r * np.cos(alpha) + 0.25
    x2 = (-theta[0] + theta[1]) / np.sqrt(2.0) + r * np.sin(alpha)
    return dict(x=np.concatenate([x1, x2], axis=0))

In [6]:
generative_model = bf.GenerativeModel(
    local_context=two_moons_context,
    global_context=None,
    prior=two_moons_prior,
    likelihood=two_moons_likelihood,
)

In [7]:
# pass batch size and steps per epoch here for now
# support this issue to fix that and move these to posterior.fit()
# https://github.com/keras-team/keras/issues/19528
train_dataset = bf.datasets.OnlineDataset(
    generative_model,
    batch_size=64,
    steps_per_epoch=100,
    workers=8,
    use_multiprocessing=True,
)
validation_dataset = bf.datasets.OnlineDataset(
    generative_model,
    batch_size=64,
    steps_per_epoch=10,
    workers=8,
    use_multiprocessing=True,
)

In [8]:
class Subnet(keras.Layer):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        self.network = keras.Sequential([
            keras.layers.Input(shape=(in_features,)),
            keras.layers.Dense(512, activation="relu"),
            keras.layers.Dense(2 * out_features),
        ])
        
    def call(self, x):
        parameters = self.network(x)
        scale, shift = tf.split(parameters, 2, axis=1)
        return dict(scale=scale, shift=shift)

In [9]:
# use a sequential coupling flow
# method name is subject to change
# we will allow to use the default BayesFlow networks in the future
inference_network = bf.networks.CouplingFlow.uniform(
    subnet_constructor=Subnet,
    # 2 parameters
    features=2,
    # 2 observables that we condition on
    conditions=2,
    layers=4,
    transform="affine",
    base_distribution="normal",
)

In [10]:
posterior = bf.AmortizedPosterior(inference_network=inference_network)

In [11]:
optimizer = keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=0.01)

In [12]:
posterior.compile(optimizer)

In [13]:
callbacks = [
    # track losses and metrics in TensorBoard
    keras.callbacks.TensorBoard()
]

In [ ]:
posterior.fit(train_dataset, validation_data=validation_dataset, epochs=100, callbacks=callbacks)

Epoch 1/100


In [ ]:
bf.diagnostics.show_posterior(posterior=posterior)